In [2]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import gensim
import time

from sklearn.decomposition import NMF, TruncatedSVD, LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances

from gensim.models.coherencemodel import CoherenceModel

#import TextCleaning

In [3]:
clean_df = pd.read_pickle('/home/kjl5t/prd/publicrd/data/prd/RND_Topic_Modelling/processed_dataset_stanford_lemma.pkl')

In [4]:
clean_df.shape

(543433, 25)

In [5]:
clean_df.columns

Index(['original index', 'PROJECT_ID', 'ABSTRACT', 'FY', 'FIRST_CHAR',
       'LAST_CHAR', 'DEPARTMENT', 'AGENCY', 'IC_CENTER', 'PROJECT_NUMBER',
       'PROJECT_TITLE', 'PROJECT_TERMS', 'CONTACT_PI_PROJECT_LEADER',
       'OTHER_PIS', 'ORGANIZATION_NAME', 'CFDA_CODE', 'FY_TOTAL_COST',
       'working_abstract', 'nchar', 'Start Char', 'Field Count',
       'lemma_docs_with_stop', 'tokened_docs_nostop', 'tns_bi_tri_docs',
       'final_tokens'],
      dtype='object')

In [6]:
clean_df.head(10)

,original index,PROJECT_ID,ABSTRACT,FY,FIRST_CHAR,LAST_CHAR,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,...,CFDA_CODE,FY_TOTAL_COST,working_abstract,nchar,Start Char,Field Count,lemma_docs_with_stop,tokened_docs_nostop,tns_bi_tri_docs,final_tokens
0,0,89996,"This is a project to explore Game-based, Metap...",2008,"This is a project to explore Game-based, Metap...",.,NSF,NSF,NaN,0814512,...,47.076,1999467.0,"This is a project to explore Game-based, Metap...",2057,T,0,"[project, explore, game, base, metaphor, enhan...","[project, explore, game, base, metaphor, enhan...","[project, explore, game, base, metaphor, enhan...","[project, explore, game, base, metaphor, enhan..."
1,1,89997,Institution: Franklin Institute Science Museum...,2008,Institution: Franklin Institute Science Museum...,.,NSF,NSF,NaN,0741659,...,47.076,1799699.0,"Institution: Science Museum PI: Snyder, Steve...",1913,I,1,"[institution, science, Museum, pi, Snyder, Ste...","[institution, science, museum, pi, steve, proj...","[institution, science, museum, pi, steve, proj...","[institution, science, museum, pi, steve, proj..."
2,2,89998,Through programs (including small group conver...,2008,Through programs (including small group conver...,.,NSF,NSF,NaN,0813522,...,47.076,1505858.0,Through programs (including small group conver...,1154,T,0,"[program, include, small, group, conversation,...","[program, small, group, conversation, citizen,...","[program, small, group, conversation, citizen,...","[program, small, group, conversation, citizen,..."
3,3,89999,In partnership with the American Chemical Soci...,2008,In partnership with the American Chemical Soci...,.,NSF,NSF,NaN,0838627,...,47.049,51000.0,In partnership with the American Chemical Soci...,826,I,0,"[partnership, American, Chemical, Society, ACS...","[partnership, american, chemical, society, acs...","[partnership, american, chemical, society, acs...","[partnership, american, chemical, society, acs..."
4,4,90000,Amphibian populations around the world are exp...,2008,Amphibian populations around the world are exp...,.,NSF,NSF,NaN,0815315,...,47.074,370996.0,Amphibian populations around the world are exp...,1322,A,1,"[amphibian, population, world, experience, dec...","[amphibian, population, world, experience, dec...","[amphibian, population, world, experience, dec...","[amphibian, population, world, experience, dec..."
5,5,90001,The Center for Molecular Interfacing (CMI) wil...,2008,The Center for Molecular Interfacing (CMI) wil...,.,NSF,NSF,NaN,0847926,...,47.049,1519821.0,The Center for Molecular Interfacing (CMI) wil...,2309,T,0,"[Center, Molecular, Interfacing, CMI, enable, ...","[center, molecular, interfacing, cmi, enable, ...","[center, molecular, interfacing, cmi, enable, ...","[center, molecular, interfacing, cmi, enable, ..."
6,6,90002,DRU: Integrated optimization of evacuation and...,2008,DRU: Integrated optimization of evacuation and...,.,NSF,NSF,NaN,0826832,...,47.075,757499.0,DRU: Integrated optimization of evacuation and...,4506,D,1,"[DRU, integrate, optimization, evacuation, she...","[dru, integrate, optimization, evacuation, she...","[dru, integrate, optimization, evacuation_shel...","[dru, integrate, optimization, evacuation_shel..."
7,7,90003,The Flora of China (FOC) is an international c...,2008,The Flora of China (FOC) is an international c...,.,NSF,NSF,NaN,0813935,...,47.074,250000.0,The (FOC) is an international collaborative p...,1255,T,0,"[FOC, international, collaborative, project, g...","[foc, international, collaborative, project, g...","[foc, international, collaborative, project, g...","[foc, international, collaborative, project, g..."
8,9,90005,The overall goal of this project is to reconst...,2008,The overall goal of this project is to reconst...,.,NSF,NSF,NaN,0823522,...,47.050,357187.0,The overall goal of this project is to reconst...,4776,T,1,"[overall, goal, project, be, reconstruct, low,...","[goal, project, reconstruct, low, frequency, b...","[goal, project, reconstruct, low